# Homework 1 &mdash; KFolds
### Connor Hornibrook
### Data Mining II &mdash; Dr. Breitzman

## Data Retrieval and Cleanup

For this project, I wanted to do something with the [2016 election data](https://www.kaggle.com/benhamner/2016-us-election) found at Kaggle. I downloaded the available SQLite database which 
contains three tables:

- primary_results &mdash; each candidate's vote share from a given county
- county_facts &mdash; table containing demographic and geographic information for counties and states, can be linked to primary_results by joining on their two respective ```FIPS``` columns.
- county_facts_dictionary &mdash; a lookup table that provides definitions for the oddly-named columns in county_facts

I wanted to make a model to guess who won a given county based on population density, age demographics, and the share of the vote they received.

To make things a little simpler as this is just a proof of concept, I decided to limit the range of values to Democratic Party results. I then had to create a view in the database that simply was comprised
of only one record per county (the winning candidate numbers), and that cleaned up some of the data. 

To ensure that this would work with an ```sklearn``` model, I eliminated any county with ```null``` age demographic information, and converted the candidate column to feature numeric ids rather than their actual names. 

The ```SQL``` for this view can be found below:

```sql
create view county_winners as
select  
       case when a.candidate = 'HILLARY CLINTON' then 1
            when a.candidate = 'BERNIE SANDERS' then 2
            when a.candidate = 'MARTIN O''MALLEY' then 3
            else 4 end candidate
     , cast(c.fips as integer) state_fips
     , cast(a.county_fips as integer) county_fips
     , cast(a.fraction_votes as float) vote_pct
     , cast(b.age135214 as float) under_5yo_pct
     , cast(b.age295214 as float) under_18yo_pct
     , cast(b.age775214 as float) over_65yo_pct
     , cast(b.pop060210 as float) pop_density
from (
	select
	    fips county_fips
	  , trim(upper(candidate)) candidate
	  , trim(upper(state)) state
	  , fraction_votes
	  , dense_rank() over (
	  		partition by fips, party
	  		order by fraction_votes desc
	  ) county_place
	from primary_results 
	where party = 'Democrat'
	and candidate not in (' No Preference', ' Uncommitted') and fips is not null
) a 
left join county_facts b on a.county_fips = b.fips
left join county_facts c on a.state = upper(trim(c.area_name))
where a.county_place = 1
  and b.age135214 is not null
  and b.age295214 is not null
  and b.age775214 is not null
  and b.pop060210 is not null
  and a.fraction_votes is not null;
;
```


## Writing the Script

Now that I had a simple database view to use (```select * from county_winners```), I could load it up via ```pandas``` and craft the needed ```numpy``` arrays for my independent and dependent variables.

The SQLite database that was used for this project is included in this repository as ```database.sqlite```.

To run this outside of this jupyter notebook, simply run ```./kfolds.py [-p, --print-scores]``` in the terminal.

In [15]:
# import needed modules
import pandas
import sqlite3
import pathlib
import pickle
import argparse
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

In [16]:
# path to the election sqlite db
_db_path = pathlib.Path(".", "database.sqlite")

# path to serialized model (if it exists)
_model_path = pathlib.Path(".", "model.bin")

In [17]:
# function that returns a Naive Bayes model, will pickle and save locally
def get_model():
    if _model_path.exists():
        with open(str(_model_path), "rb") as f:
            model = pickle.load(f)
    else:
        model = MultinomialNB()
        with open(str(_model_path), "wb") as f:
            pickle.dump(model, f)
    return model

In [18]:
# function that reads the election data from sqllite, determines the dependent
# and independent variables, and scales the actual values
def get_data():
    scaler = MinMaxScaler(feature_range=(0,1))
    with sqlite3.connect(_db_path) as conn:
        raw = pandas.read_sql_query(
            "select * from county_winners",
            conn
        )
        y = raw.candidate.to_numpy()
        x = raw.drop(columns="candidate").to_numpy()
        x = scaler.fit_transform(x)
    return x, y

In [19]:
# encapsulated logic for using KFolds cross validation on
# a model, using the input independent/dependent variables.
# returns the accuracy scores, and optionally prints them
def get_accuracy_scores(x, y, model, print_scores=False):
    kf = KFold(n_splits=10)
    accuracy_scores = []
    for train_i, test_i in kf.split(x):
        x_train, x_test, y_train, y_test = x[train_i], x[test_i], y[train_i], y[test_i]
        model.fit(x_train, y_train)
        score = model.score(x_test, y_test)
        accuracy_scores.append(score)
        if print_scores:
            print(score)
    return accuracy_scores

In [20]:
# put it all together
def main():
    x, y = get_data()
    get_accuracy_scores(
        x, y, get_model(), print_scores=True
    )

In [21]:
# run it
main()

0.7385159010600707
0.7773851590106007
0.35335689045936397
0.45936395759717313
0.5547703180212014
0.5441696113074205
0.4664310954063604
0.8546099290780141
0.8439716312056738
0.3120567375886525
